In [1]:
# install syft if we are on google colab
%load_ext autoreload
%autoreload 2
import syft as sy
print(f"You're running syft version: {sy.__version__}")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from tqdm import tqdm
import time

/home/azureuser/.local/share/virtualenvs/python39-2vM0bQN-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You're running syft version: 0.7.0-beta.21


In [2]:
domain_client = sy.login(email='info@openmined.org', password='changethis',port = 8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!


In [3]:
data = domain_client.datasets[-1]

X_train = data["train_images"]
Y_train = data["train_labels"]

X_dev = data["train_images"]
Y_dev = data["train_labels"]

m,n = X_train.public_shape

X_train = X_train.T
X_train = X_train *(1/ 255.0)

X_dev = X_dev.T
X_dev = X_dev *(1/ 255.0)


**Warning**: Blob Storage is disabled on this client node. Switching to database store.

**Warning**: Blob Storage is disabled on this client node. Switching to database store.


In [4]:
from syft.core.node.common.node_service.user_manager.user_messages import (
    UpdateUserMessage,
)

# Upgrade admins budget
content = {"user_id": 1, "budget": 9_999_999}
domain_client._perform_grid_request(grid_msg=UpdateUserMessage, content=content)

domain_client.privacy_budget

9999999.0

In [5]:
def init_params(input_size: int):
    print(f"Using input size: {input_size}")
    W1 = np.random.rand(6, input_size) - 0.5
    b1 = np.random.rand(6, 1) - 0.5
    W2 = np.random.rand(6, 6) - 0.5
    b2 = np.random.rand(6, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return Z*(Z>0)

def softmax(Z):
    exp_cache = Z.exp()
    inv =  (exp_cache.sum().reciprocal())
    
    A = exp_cache * inv
    return A
    
def forward_prop(W1, b1, W2, b2, X):    
    Z1 = X.__rmatmul__(W1) + b1
    A1 = ReLU(Z1)
    Z2 = A1.__rmatmul__(W2) + b2
    A2 = softmax(Z2) 
    return Z1, A1, Z2, A2


def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size),Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = Y.one_hot()
    dZ2 = A2 - one_hot_Y
    dW2 = dZ2@(A1.T) * (1/m)
    db2 = dZ2.sum() * (1/m)
    dZ1 = dZ2.__rmatmul__(W2.T) * ReLU_deriv(Z1)
    dW1 = dZ1@(X.T) * (1/m)
    db1 = dZ1.sum()*(1/m)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = (dW1 * alpha - W1) * -1
    b1 = (db1 * alpha - b1) * -1
    W2 = (dW2 * alpha - W2) * -1
    b2 = (db2 * alpha - b2) * -1
    return W1, b1, W2, b2


def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params(X.public_shape[0])
    print("[INFO]: Starting training!\n")
    for i in tqdm(range(iterations)):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)

        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        # if i % 40 == 0:
        #     print("Predicition will be availabe in the next iteration.....Thank you for your support.")
        #     # predictions = get_predictions(A2)
        #     # print("Iteration: ", i, " Accuracy : " ,get_accuracy(predictions, Y))

    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 1)

In [10]:
for ptr in [W1, b1, W2, b2]:
    ptr.block_with_timeout(60)

In [11]:
print(W1.exists)
print(b1.exists)
print(W2.exists)
print(b2.exists)

True
True
True
True


In [ ]:
# DEBUG CELLS
# W1_O = W1.get_copy()
# print(type(W1_O))

# b1_O = b1.get_copy()
# print(type((b1_O.child)))

# W2_O = W2.get_copy()
# print(type((W2_O.child)))

# b2_O = b2.get_copy()
# print(type((b2_O.child)))


In [14]:
W1 = W1.publish(sigma=1000)
b1 = b1.publish(sigma=1000)
W2 = W2.publish(sigma=1000)
b2 = b2.publish(sigma=1000)

In [13]:
a,b,c,d = init_params(X_train.public_shape[0])
print(a.shape)
print(b.shape)
print(c.shape)
print(d.shape)

Using input size: 4096
(6, 4096)
(6, 1)
(6, 6)
(6, 1)


In [15]:
t1,t2,t3,t4 = W1.get_copy(),b1.get_copy(),W2.get_copy(),b2.get_copy()

In [16]:
print(t1.shape)
print(t2.shape)
print(t3.shape)
print(t4.shape)

(6, 4096)
(6,)
(0,)
(6,)


In [20]:
t1

array([[-4.03931907e+08,  2.70095885e+08, -2.03099239e+09, ...,
         2.24026917e+09, -8.21211127e+07,  3.52926429e+08],
       [-3.38859132e+08, -2.47115319e+08, -2.35271299e+08, ...,
        -1.94550042e+08, -1.37187998e+08, -2.98735239e+08],
       [-1.05739749e+09, -1.42284317e+09, -1.91803234e+08, ...,
         1.22387712e+09,  1.97518672e+09, -1.01735474e+09],
       [-1.85626899e+09,  2.50373817e+08, -1.67554057e+09, ...,
         2.54363362e+09,  2.01326087e+09,  1.90351145e+09],
       [-1.85269743e+09,  2.25392655e+09, -8.82008150e+08, ...,
        -9.14471869e+08,  1.10946054e+09,  2.61804698e+08],
       [-2.14396650e+09,  7.59615605e+08,  1.43420774e+09, ...,
        -2.48304895e+09,  1.39128885e+09, -2.23387462e+09]])